In [10]:
import sys
import os
import requests
import concurrent.futures
import unittest

# Add the parent directory of the 'src' directory to the Python path
from sglang.srt.server import Runtime as SGLangServer
from sglang.srt.managers.io_struct import SchedulingMetricsReqInput, MigrationReq

sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''), "../multi_node/")))

In [11]:
server_1_url = "http://127.0.0.1:30000"
server_2_url = "http://127.0.0.1:10007"

In [12]:
add_req_url = f'{server_1_url}/add_request'
r = requests.post(add_req_url, json={ 
    "text": "Say this is a warmup request.",
    "sampling_params": {
        "temperature": 0,
        "max_new_tokens": 16,
    },
})
r

<Response [200]>

In [13]:
metrics_url = f"{server_1_url}/scheduling_metrics"
metrics_results1 = requests.post(metrics_url, json={"prompt": "sample prompt"})
metrics_data1 = metrics_results1.json()
metrics_data1

{'waiting_queue_len': 0,
 'running_req_len': 0,
 'prefix_match_len': 0,
 'token_kv_available_size': 198516,
 'evicatable_size': 0,
 'tree_cache_metrics_hit': 0,
 'tree_cache_metrics_total': 0,
 'input_len': 3,
 'total_radix_cache_processing_time': 0.04487252235412598,
 'queue_processing_time': 0.0010666847229003906,
 'inner_router_time': 0.044934988021850586,
 'matching_overhead': 0.06222724914550781,
 'tokenization_time': 0.00020694732666015625,
 'routing_time': 0.00016689300537109375,
 'return_time': 4.76837158203125e-06,
 'request_processing_time': 5.698204040527344e-05,
 'total_internal_request_time': 0.04705643653869629}

In [14]:
import zmq
import zmq.asyncio

In [6]:
context = zmq.asyncio.Context()
recv_from_rounter = context.socket(zmq.PULL)
recv_from_rounter.bind(f"tcp://127.0.0.1:16372")

send_to_rounter = context.socket(zmq.PUSH)
send_to_rounter.connect(f"tcp://localhost:10012")

<SocketContext(connect='tcp://localhost:10012')>

In [7]:
migrate_url = f"{server_1_url}/migrate_control"
requests.post(migrate_url, params={"migration_target_url": "tcp://127.0.0.1:16372"})
mreq = await recv_from_rounter.recv_pyobj()

In [8]:
mreq.requets

[rid(n=ea674582db1245db9619b6a69099e894, input_ids=[1, 15753, 456, 349, 264, 6100, 715, 2159, 28723], ]

In [9]:
send_to_rounter.send_pyobj(mreq)

<Future finished result=None>

In [28]:
migrate_url = f"{server_2_url}/migrate_control"
requests.post(migrate_url, params={"migration_target_url": "tcp://127.0.0.1:16372"})
mreq2 = await recv_from_rounter.recv_pyobj()

In [15]:
migrate_url = f"{server_1_url}/migrate_control"
requests.post(migrate_url, params={"migration_target_url": "tcp://localhost:10012"})

<Response [200]>

In [ ]:
migrate_url = f"{server_2_url}/migrate_control"
requests.post(migrate_url, params={"migration_target_url": "tcp://localhost:10006"})